In [1]:
import pandas as pd
from scipy.stats import ttest_ind
import numpy as np

import re

In [2]:
config_prefix = ['TI', 'TR', 'BR', 'BI']

config_algo_names = ['BinarySearch', 'BubbleSort', 'Factorial', 'Fibonacci', 'IntegerBinary', 'MultiplyMatrix', 'PrimeFactors', 'ReverseString']

config_id_variable = "CASE"

config_answer_variables = [
    ['TI21_01', 'TR21_01', 'BR21_01', 'BI21_01'],
    ['TI22_01', 'TR22_01', 'BR22_01', 'BI22_01'],
    ['TI23_01', 'TR23_01', 'BR23_01', 'BI23_01'],
    ['TI24_01', 'TR24_01', 'BR24_01', 'BI24_01'],
    ['TI25_01', 'TR25_01', 'BR25_01', 'BI25_01'],
    ['TI26_01', 'TR26_01', 'BR26_01', 'BI26_01'],
    ['TI27_01', 'TR27_01', 'BR27_01', 'BI27_01'],
    ['TI28_01', 'TR28_01', 'BR28_01', 'BI28_01'],
]

config_idk_variables = [
    ['TI21_01a', 'TR21_01a', 'BR21_01a', 'BI21_01a'],
    ['TI22_01a', 'TR22_01a', 'BR22_01a', 'BI22_01a'],
    ['TI23_01a', 'TR23_01a', 'BR23_01a', 'BI23_01a'],
    ['TI24_01a', 'TR24_01a', 'BR24_01a', 'BI24_01a'],
    ['TI25_01a', 'TR25_01a', 'BR25_01a', 'BI25_01a'],
    ['TI26_01a', 'TR26_01a', 'BR26_01a', 'BI26_01a'],
    ['TI27_01a', 'TR27_01a', 'BR27_01a', 'BI27_01a'],
    ['TI28_01a', 'TR28_01a', 'BR28_01a', 'BI28_01a'],
]

config_time_variables = [
    ['TIME011','TIME083','TIME035','TIME059'],
    ['TIME038','TIME014','TIME062','TIME086'],
    ['TIME065','TIME041','TIME086','TIME017'],
    ['TIME092','TIME068','TIME020','TIME044'],
    ['TIME023','TIME095','TIME047','TIME071'],
    ['TIME050','TIME026','TIME074','TIME098'],
    ['TIME077','TIME053','TIME0101','TIME029'],
    ['TIME0104','TIME080','TIME032','TIME056'],
]



config_answer_patterns = [
    ['.*3.*', '.*3.*', '.*3.*', '.*3.*'],
    ['.*3.*16.*23.*42.*61.*75.*536.*','.*3.*16.*23.*42.*61.*75.*536.*','.*3.*16.*23.*42.*61.*75.*536.*','.*3.*16.*23.*42.*61.*75.*536.*'],
    ['.*6.*','.*6.*','.*120.*','.*120.*'],
    ['.*2.*', '.*2.*','.*2.*','.*2.*'],
    ['.*1.*0.*0.*0.*1.*','.*1.*0.*0.*0.*1.*','.*1.*0.*0.*0.*1.*','.*1.*0.*0.*0.*1.*'],
    ['.*6.*6.*6.*12.*12.*12.*18.*18.*18.*','.*6.*6.*6.*12.*12.*12.*18.*18.*18.*','.*6.*6.*6.*12.*12.*12.*18.*18.*18.*','.*6.*6.*6.*12.*12.*12.*18.*18.*18.*'],
    ['.*3.*5.*','.*3.*5.*','.*3.*5.*','.*3.*5.*'],
    ['.*gnikcar[tT].*ey[eE].*','.*gnikcar[tT].*ey[eE].*','.*gnikcar[tT].*ey[eE].*','.*gnikcar[tT].*ey[eE].*']
]


config_datasheet = r'./data_DG.xlsx'

In [3]:
df_raw = pd.read_excel(config_datasheet)
df_raw.head(5)

,CASE,SERIAL,REF,QUESTNNR,MODE,STARTED,TI21_01,TI21_01a,TI49_01,TI49_01a,...,MAILSENT,LASTDATA,FINISHED,Q_VIEWER,LASTPAGE,MAXPAGE,MISSING,MISSREL,TIME_RSI,DEG_TIME
0,165,NaN,NaN,qnrNew,interview,2022-06-20 09:31:20,NaN,NaN,NaN,2.0,...,NaN,2022-06-20 09:39:01,1,0,106,106,0.0,0.0,2.42,251
1,175,NaN,NaN,qnrNew,interview,2022-06-20 10:23:06,NaN,NaN,NaN,2.0,...,NaN,2022-06-20 10:29:10,1,0,106,106,2.0,0.0,2.61,265
2,182,NaN,NaN,qnrNew,interview,2022-06-20 11:02:59,NaN,NaN,1,1.0,...,NaN,2022-06-20 11:10:07,1,0,106,106,0.0,0.0,2.61,254
3,198,NaN,NaN,qnrNew,interview,2022-06-20 16:43:42,Arooba,1.0,NaN,2.0,...,NaN,2022-06-20 16:48:46,1,0,106,106,2.0,2.0,2.67,291
4,205,NaN,NaN,qnrNew,interview,2022-06-21 13:19:04,NaN,NaN,NaN,NaN,...,NaN,2022-06-21 13:19:10,0,0,1,1,NaN,NaN,0.50,0


In [4]:
df_matrix = []

df_total = pd.DataFrame([], columns=["case", "factor", "algorithm", "answer", "idk", "time"])
# partion the data into the factors
for algorithm_idx, algorithm in enumerate(config_algo_names):
    df_array = []
    for factor_idx, factor in enumerate(config_prefix):
        df = pd.DataFrame(df_raw, columns = [config_id_variable,  config_answer_variables[algorithm_idx][factor_idx], config_time_variables[algorithm_idx][factor_idx], config_idk_variables[algorithm_idx][factor_idx]])
        df.columns = ["case", "answer", "time", "idk"]
        df = df[df['time'].notna()]
        df["factor"] = factor
        df["algorithm"] = algorithm
        df_total = pd.concat([df_total, df])

df_total["idk"] = df_total["idk"].apply(lambda x: x != 1.0)
df_total[df_total["answer"].isna()]

,case,factor,algorithm,answer,idk,time
0,165,BI,BubbleSort,NaN,True,4.0
1,175,TI,Factorial,NaN,False,34.0
7,210,TR,Factorial,NaN,False,41.0
0,165,BR,Factorial,NaN,True,4.0
8,211,BI,Factorial,NaN,False,71.0
0,165,TI,Fibonacci,NaN,True,29.0
3,198,BR,Fibonacci,NaN,False,31.0
20,235,BR,Fibonacci,NaN,False,32.0
9,212,BI,Fibonacci,NaN,True,23.0
0,165,TR,IntegerBinary,NaN,True,5.0


In [5]:
# potential query
df_total[(df_total["factor"]=="TI") & (df_total["algorithm"] == "BinarySearch")]

,case,factor,algorithm,answer,idk,time
3,198,TI,BinarySearch,Arooba,False,8.0
8,211,TI,BinarySearch,3,False,58.0
14,224,TI,BinarySearch,3,False,30.0
17,231,TI,BinarySearch,3,False,85.0
18,232,TI,BinarySearch,3,False,5.0
20,235,TI,BinarySearch,2,False,118.0
25,240,TI,BinarySearch,3,False,84.0


In [6]:
def is_correct(row):
    algorithm = row["algorithm"]
    factor = row["factor"]
    algo_idx = config_algo_names.index(algorithm)
    factor_idx = config_prefix.index(factor)
    pattern = config_answer_patterns[algo_idx][factor_idx]
    regex = re.compile(pattern)
    answer = str(row["answer"])
    result = regex.match(str(answer))
    return result is not None

df_total["correctness"] = df_total.apply(lambda x: is_correct(x), axis=1)
df_total

,case,factor,algorithm,answer,idk,time,correctness
3,198,TI,BinarySearch,Arooba,False,8.0,False
8,211,TI,BinarySearch,3,False,58.0,True
14,224,TI,BinarySearch,3,False,30.0,True
17,231,TI,BinarySearch,3,False,85.0,True
18,232,TI,BinarySearch,3,False,5.0,True
...,...,...,...,...,...,...,...
2,182,BI,ReverseString,Arooba,False,4.0,False
7,210,BI,ReverseString,gnikcarTeyE,False,72.0,True
9,212,BI,ReverseString,gnikcarTeyE,False,60.0,True
15,226,BI,ReverseString,"""gnikcarTeyE""",False,29.0,True


In [7]:
df_total.to_excel("./results/preprocessed.xlsx", index=False)